Diff PRE-POST

In [ ]:
from scipy.stats import gaussian_kde

def diff_size(nonPicked_ratio, all_measures, nonPicked_measures):
    picked_ratio = 1 - nonPicked_ratio

    # By miu,sigma , assume normalization
    miu_all = all_measures.mean()
    sigma_all = all_measures.std()
    miu_nonPicked = nonPicked_measures.mean()
    sigma_nonPicked = nonPicked_measures.std()
    miu_picked = (1 / picked_ratio) * miu_all - (nonPicked_ratio / picked_ratio) * miu_nonPicked
    sigma_picked = np.sqrt((1 / picked_ratio) ** 2 * sigma_all ** 2 - (nonPicked_ratio / picked_ratio) ** 2 * sigma_nonPicked ** 2)

    # By kde
    x_values = np.linspace(all_measures.min(), all_measures.max(), num=int(all_measures.max() - all_measures.min()))
    kde_all = gaussian_kde(all_measures)(x_values)
    kde_nonPicked = gaussian_kde(nonPicked_measures)(x_values)
    kde_picked = (1 / picked_ratio) * kde_all - (nonPicked_ratio / picked_ratio) * kde_nonPicked
    kde_miu = np.sum(kde_picked * x_values)
    kde_sigma = np.sqrt(np.sum(kde_picked * (x_values - kde_miu) ** 2))

    # By hist
    hist_all, bins, p = plt.hist(all_measures,density=True)
    hist_nonPicked = plt.hist(nonPicked_measures, density=True,bins=bins)[0]
    hist_picked = (1 / picked_ratio) * hist_all - (nonPicked_ratio / picked_ratio) * hist_nonPicked
    hist_picked = hist_picked/ np.sum(hist_picked)
    bins = [(var+ bins[i + 1])/2 for i, var in enumerate(bins) if i+1 != len(bins)]
    hist_miu = np.sum(hist_picked * bins)
    hist_sigma = np.sqrt(np.sum(hist_picked * (bins - hist_miu) ** 2))

    return (miu_picked, sigma_picked), (kde_miu, kde_sigma), (hist_miu, hist_sigma)

Debug counter

In [ ]:
import os
os.chdir('/home/yotam/FruitSpec/Code/fsCounter')
print(os.getcwd())
# from vision.data.results_collector import ResultsCollector
from vision.tools.manual_slicer import slice_to_trees
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
color_dict = {"bin1": [300, 15], "bin2": [15, 45], "bin3": [45, 90], "bin4": [90, 165]}


def filter_logic(path, analysis='counter'):
    df = pd.read_csv(os.path.join(path, 'measures.csv'))

    if analysis == 'size':
        df_res = df[(df['width'] < 90) & (df['height'] < 90)]
        # df_res = df[df.apply(lambda x: False if (x.width / x.height > 1.25) or (x.width / x.height < 0.75) else True, axis=1)]
    elif analysis == 'color':
        df_res = df[(df['color'] < 165) & (df['color'] > 300)]
    else:
        df_res = df
    return df_res

In [ ]:
def trackers_into_plot(df_tree, df_res, analysis='counter'):
    df_tree['end'].replace([-1], math.inf,inplace=True)
    plot_det = []
    for frame_id, df_frame in df_res.groupby('frame'):
        limit = df_tree[df_tree['frame_id'] == frame_id]
        if limit.empty:
            continue
        limit = limit.iloc[0]
        df = df_frame[(df_frame['x1'] > limit['start']) & (df_frame['x1'] < limit['end'])]
        plot_det.append(df)
    if analysis == 'counter':
        var = len(pd.concat(plot_det,axis=0)['track_id'].unique())
    elif analysis == 'size':
        df_group = pd.concat(plot_det,axis=0).groupby('track_id')
        print(len(df_group))
        var = df_group.apply(lambda x : x.width.mean() if x.width.mean() > x.height.mean() else x.height.mean())
    return var

In [ ]:
def run_counter(scan,output):
    df_sum = pd.DataFrame()
    for row in os.listdir(scan):
        row_folder = os.path.join(scan,row)
        if not row.__contains__('TiltDown'):
            continue
        df_res = filter_logic(row_folder)
        trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]),None,None)
        fruits_num = trackers_into_plot(trees_slices,df_res,analysis='counter')
        df_sum = df_sum.append({'repeat':row, 'fruits_num':fruits_num},ignore_index = True)
    df_sum.to_csv(output,index=True,header=True,mode='w')
    return df_sum

In [ ]:
scan_folder = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222'
output_path = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222/results.csv'
df_sum_counter = run_counter(scan_folder,output_path)

Debug size

In [ ]:
def run_size(scan,output):
    for index, row in enumerate(os.listdir(scan)):
        row_folder = os.path.join(scan,row)
        if not row.__contains__('TiltDown40cm') or not row.__contains__('1st'):
            continue
        df_res = filter_logic(row_folder,analysis='size')
        trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]),None,None)
        size_df = trackers_into_plot(trees_slices,df_res,analysis='size')
        count = trackers_into_plot(trees_slices,df_res,analysis='counter')
        if row.__contains__('pre'):
            plt.hist(size_df)
            plt.ylim(0,15)
            pre_count = count
            pre_measures = size_df
        if row.__contains__('post'):
            plt.hist(size_df)
            plt.ylim(0,15)
            post_count =count
            post_measures = size_df
        plt.title(row)
        plt.show()
    print(diff_size(1 -pre_count/post_count, pre_measures,post_measures))

In [ ]:
scan_folder = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222'
output_path = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222/results.csv'
run_size(scan_folder,output_path)

Debug Color

In [ ]:
def run_color():
    for index, row in enumerate(os.listdir(scan)):
        row_folder = os.path.join(scan,row)
        if not row.__contains__('TiltDown40cm') or not row.__contains__('2nd'):
            continue
        df_res = filter_logic(row_folder,analysis='color')
        trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]),None,None)
        color_df = trackers_into_plot(trees_slices,df_res,analysis='color')
        count = trackers_into_plot(trees_slices,df_res,analysis='counter')
        if row.__contains__('pre'):
            plt.hist(size_df)
            pre_count = count
            pre_measures = color_df
        if row.__contains__('post'):
            plt.hist(size_df)
            post_count =count
            post_measures = color_df
        plt.title(row)
        plt.show()

    print(diff_size(1-post_count/pre_count, all_measures= pre_measures,nonPicked_measures=post_measures))



In [ ]:
df = pd.read_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\061022\color_for_hist_2676.csv')
df['color_median'] = None
df['color_mean'] = None
for j, row in df.iterrows():
    try:
        hue_med,hue_mean = get_color(plot_bbox(row, (0, j)), index=row['frame'])
        df.at[j, 'color_median'] = hue_med
        df.at[j, 'color_mean'] = hue_mean
    except Exception as e:
        print(repr(e))
df.to_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\061022\color_for_hist_2676.csv', index=False)